<h1>Imports</h1>

In [34]:
# Importing necessary packages

import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<h1>Supplementary Functions</h1>

In [2]:
# Calculating angle between joints

def calculate_angle(point1, point2, point3):
    point1 = np.array(point1)
    point3 = np.array(point3)
    point2 = np.array(point2)
    
    radians = np.arctan2(point3[1] - point2[1], point3[0] - point2[0]) - np.arctan2(point1[1] - point2[1], point1[0] - point2[0])
    angle = np.abs((radians * 180.0)/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

<h2>Analyzing Trainer footage to set state thresholds based on Knee angles</h2>

In [3]:
state1_angle_threshold = [0, 0]
state2_angle_threshold = [0, 0]
state3_angle_threshold = [0, 0]

In [137]:
trainer_footage = "sample_videos/Abdullah_Umar.mp4"
client_footage = './sample_videos/AbdulRehman_Umar.mp4'

In [5]:
cap = cv2.VideoCapture(trainer_footage)

knee_angles = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
                     
            
            # Knee Angle Calculation
            
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
                knee_angles.append(knee_angle)
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                knee_angles.append(knee_angle)
                        
            
            
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    
        
# print min and max angles for elbow

print("Knee Angle: ", min(knee_angles), max(knee_angles))

# Set the threshold values for each state

state3_angle_threshold[0] = min(knee_angles)
state1_angle_threshold[1] = max(knee_angles)

mid_angle = round((state3_angle_threshold[0] + state1_angle_threshold[1])/2.0, 1)

state_mid_1 = round((state3_angle_threshold[0] + mid_angle)/2.0, 1)
state_mid_2 = round((mid_angle + state1_angle_threshold[1])/2.0, 1)

state3_angle_threshold[1] = state_mid_1
state2_angle_threshold[0] = state_mid_1
state2_angle_threshold[1] = state_mid_2
state1_angle_threshold[0] = state_mid_2

Elbow Angle:  49.3 171.4


In [6]:
# print the threshold values for each state

print("State 1: ", state1_angle_threshold)
print("State 2: ", state2_angle_threshold)
print("State 3: ", state3_angle_threshold)

State 1:  [140.9, 171.4]
State 2:  [79.8, 140.9]
State 3:  [49.3, 79.8]


<p>Using knee angles will be better to transition through states.</p>
<p>This is because shoulders and elbows barely move throughout the movement</p>

<h2>Getting Angles from Trainer Footage</h2>

In [7]:
trainer_hip_angle_state1 = []
trainer_knee_angle_state1 = []

trainer_hip_angle_state2 = []
trainer_knee_angle_state2 = []

trainer_hip_angle_state3 = []
trainer_knee_angle_state3 = []

In [8]:
cap = cv2.VideoCapture(trainer_footage)

state = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
                
            
            # Knee Angle Calculation and state selection
            
            knee_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            
            
            if knee_angle >= state1_angle_threshold[0] and knee_angle <= state1_angle_threshold[1]:
                state = 1
            elif knee_angle >= state2_angle_threshold[0] and knee_angle <= state2_angle_threshold[1]:
                state = 2
            elif knee_angle >= state3_angle_threshold[0] and knee_angle <= state3_angle_threshold[1]:
                state = 3
            else:
                state = 0
                    
            if state == 1:
                trainer_knee_angle_state1.append(knee_angle.round(1))
            elif state == 2:
                trainer_knee_angle_state2.append(knee_angle.round(1))
            elif state == 3:
                trainer_knee_angle_state3.append(knee_angle.round(1))
            
                
                
            # Hip Angle Calculation
            
            hip_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                
            if state == 1:
                trainer_hip_angle_state1.append(hip_angle.round(1))
            elif state == 2:
                trainer_hip_angle_state2.append(hip_angle.round(1))
            elif state == 3:
                trainer_hip_angle_state3.append(hip_angle.round(1))
            
            
            
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

In [9]:
# print all trainer angle arrays

print("State 1:-\n")
print("Trainer Hip Angle State 1: ", trainer_hip_angle_state1)
print("Trainer Knee Angle State 1: ", trainer_knee_angle_state1)

print("\nState 2:-\n")
print("Trainer Hip Angle State 2: ", trainer_hip_angle_state2)
print("Trainer Knee Angle State 2: ", trainer_knee_angle_state2)

print("\nState 3:-\n")
print("Trainer Hip Angle State 3: ", trainer_hip_angle_state3)
print("Trainer Knee Angle State 3: ", trainer_knee_angle_state3)


State 1:-

Trainer Elbow Angle State 1:  [163.4, 163.8, 164.6, 164.7, 164.8, 164.6, 164.2, 163.6, 163.0, 162.2, 160.4, 159.3, 156.4, 152.1, 143.9, 142.5, 149.8, 156.4, 162.0, 168.7, 171.4, 170.5, 169.6, 169.2, 169.6, 169.9, 170.0, 170.1, 170.1, 170.1, 169.9, 167.6, 162.8, 157.2, 151.1, 142.3, 141.6, 148.0, 154.0, 159.7, 166.0, 169.6, 170.3, 169.9, 171.0, 171.3, 171.2, 171.1, 171.1, 170.9, 170.7, 170.2, 168.1, 163.8, 158.8, 153.3, 145.9, 142.7, 149.5, 155.8, 161.0, 167.0, 170.2, 170.6, 169.7, 169.9, 170.0, 169.8, 169.7, 169.6, 169.5, 168.2, 164.7, 159.4, 153.6, 145.3, 142.2, 148.7, 154.2, 160.4, 166.4, 170.3, 171.1, 171.0]
Trainer Shoulder Angle State 1:  [53.3, 53.6, 54.6, 54.4, 54.7, 54.6, 54.4, 54.1, 53.8, 53.5, 52.9, 53.1, 53.6, 52.9, 50.3, 34.7, 37.9, 40.2, 43.5, 45.4, 47.1, 48.4, 49.1, 50.0, 50.8, 50.9, 51.1, 51.7, 51.8, 51.7, 51.5, 50.6, 48.8, 47.1, 42.8, 38.7, 34.2, 38.2, 40.5, 41.6, 45.5, 48.2, 50.7, 51.8, 52.7, 52.4, 52.1, 52.6, 52.2, 52.5, 52.1, 51.5, 50.3, 48.8, 48.1, 45.5, 

In [10]:
# print min and max angles for each joint

print("Min and max of angles:-\n")
print("\nState 1:-\n")
print("Hip Angle: ", min(trainer_hip_angle_state1), max(trainer_hip_angle_state1))
print("Knee Angle: ", min(trainer_knee_angle_state1), max(trainer_knee_angle_state1))

print("\nState 2:-\n")
print("Hip Angle: ", min(trainer_hip_angle_state2), max(trainer_hip_angle_state2))
print("Knee Angle: ", min(trainer_knee_angle_state2), max(trainer_knee_angle_state2))

print("\nState 3:-\n")
print("Hip Angle: ", min(trainer_hip_angle_state3), max(trainer_hip_angle_state3))
print("Knee Angle: ", min(trainer_knee_angle_state3), max(trainer_knee_angle_state3))

# put min max in variables

hip_angle_state1_min = min(trainer_hip_angle_state1)
hip_angle_state1_max = max(trainer_hip_angle_state1)

knee_angle_state1_min = min(trainer_knee_angle_state1)
knee_angle_state1_max = max(trainer_knee_angle_state1)


hip_angle_state2_min = min(trainer_hip_angle_state2)
hip_angle_state2_max = max(trainer_hip_angle_state2)

knee_angle_state2_min = min(trainer_knee_angle_state2)
knee_angle_state2_max = max(trainer_knee_angle_state2)


hip_angle_state3_min = min(trainer_hip_angle_state3)
hip_angle_state3_max = max(trainer_hip_angle_state3)

knee_angle_state3_min = min(trainer_knee_angle_state3)
knee_angle_state3_max = max(trainer_knee_angle_state3)

Min and max of angles:-


State 1:-

Elbow Angle:  141.6 171.4
Shoulder Angle:  34.2 54.7
Hip Angle:  161.6 180.0
Knee Angle:  169.9 180.0

State 2:-

Elbow Angle:  82.4 136.9
Shoulder Angle:  1.4 45.5
Hip Angle:  162.9 179.8
Knee Angle:  170.5 178.6

State 3:-

Elbow Angle:  49.3 79.0
Shoulder Angle:  1.6 28.4
Hip Angle:  164.9 176.8
Knee Angle:  172.9 179.9


<h1>Comparing trainer skeleton to user and evaluating</h1>

In [138]:
# Variables

leniency = 23
errors = []
error_bool = False
depth_bool = False
knee_angle = 0
shoulder_angle = 0
hip_angle = 0
knee_angle = 0
reps = 0


# Test Variables

client_incorrect = []
trainer_incorrect = []

In [139]:
cap = cv2.VideoCapture(client_footage)

state = 0
states_visited = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            error_bool = False
            
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
            
            
            # Knee Angle Calculation
            
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            else:
                knee_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
             
           
           # State Selection
           
            if knee_angle >= state1_angle_threshold[0] and knee_angle <= state1_angle_threshold[1]:
                state = 1
                states_visited.append(1)
                
                if len(states_visited) == 0:
                    states_visited.append(1)
                elif states_visited.__contains__(1) and states_visited.__contains__(2) and states_visited.__contains__(3):
                    states_visited = []
                    states_visited.append(1)
                    depth_bool = False
                    reps += 1
                    print("Reps: ", reps)
                elif states_visited.__contains__(1) and states_visited.__contains__(2):
                    states_visited = []
                    errors.append("State 3 not visited, hit more depth")
                    error_bool = True
                    depth_bool = True
                    state = 0
                    client_incorrect.append("Full range of motion not performed")
                    trainer_incorrect.append("Full range of motion not performed")
                    reps += 1
                    print("Reps: ", reps)
                    
                
            elif knee_angle >= state2_angle_threshold[0] and knee_angle <= state2_angle_threshold[1]:
                state = 2
                states_visited.append(2)
                
            elif knee_angle >= state3_angle_threshold[0] and knee_angle <= state3_angle_threshold[1]:
                state = 3
                states_visited.append(3)
            else:
                state = 0
                states_visited.append(0)
            
                
                
            # Hip Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                
            
            # Checking Angles according to state
            
            
            trainer_hip_min = 0
            trainer_hip_max = 0
            
            trainer_knee_min = 0
            trainer_knee_max = 0
            
            if state == 1:
                
                trainer_hip_min = hip_angle_state1_min
                trainer_hip_max = hip_angle_state1_max
                
                trainer_knee_min = knee_angle_state1_min
                trainer_knee_max = knee_angle_state1_max

                
            elif state == 2:
                
                trainer_hip_min = hip_angle_state2_min
                trainer_hip_max = hip_angle_state2_max
                
                trainer_knee_min = knee_angle_state2_min
                trainer_knee_max = knee_angle_state2_max
                
            elif state == 3:
                
                trainer_hip_min = hip_angle_state3_min
                trainer_hip_max = hip_angle_state3_max
                
                trainer_knee_min = knee_angle_state3_min
                trainer_knee_max = knee_angle_state3_max
                
            else:
                
                trainer_hip_min = 0
                trainer_hip_max = 0
                
                trainer_knee_min = 0
                trainer_knee_max = 0
            
            
            # Perform checking of errors when user is not in state 0
            
            if state != 0: 
                
                
                # Hip Angle Matching
                
                if hip_angle < (trainer_hip_min - leniency):
                    errors.append("Hip Angle too low")
                    error_bool = True
                    client_incorrect.append(hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_min - leniency)
                elif hip_angle > (trainer_hip_max + leniency):
                    errors.append("Hip Angle too high")
                    error_bool = True
                    client_incorrect.append(hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_max + leniency)


                # Knee Angle Matching
                
                if knee_angle < (trainer_knee_min - leniency):
                    errors.append("Knee Angle too low")
                    error_bool = True
                    client_incorrect.append(knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_min - leniency)
                elif knee_angle > (trainer_knee_max + leniency):
                    errors.append("Knee Angle too high")
                    error_bool = True
                    client_incorrect.append(knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_max + leniency)
                
            
        except:
            pass 
        
        if depth_bool == True:
            cv2.putText(image, "Increase Depth", (600, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
        else:
            cv2.putText(image, "Depth Perfect", (600, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
                    
        if error_bool == True:
            cv2.putText(image, "Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
            
        else:
            cv2.putText(image, "No Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
        
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

Reps:  1
Reps:  2
Reps:  3
Reps:  4
Reps:  5


In [140]:
for i in range(len(errors)):
    print(errors[i] + " => Client: " + str(client_incorrect[i]) + " vs Trainer: " + str(trainer_incorrect[i]))

Hip Angle too low => Client: 140.4 vs Trainer: 141.6
Knee Angle too low => Client: 136.0 vs Trainer: 149.9
Knee Angle too low => Client: 132.3 vs Trainer: 149.9
Knee Angle too low => Client: 132.3 vs Trainer: 149.9
Knee Angle too low => Client: 131.5 vs Trainer: 149.9
Hip Angle too low => Client: 141.0 vs Trainer: 141.6
Knee Angle too low => Client: 130.9 vs Trainer: 149.9
Hip Angle too low => Client: 141.2 vs Trainer: 141.6
Knee Angle too low => Client: 130.7 vs Trainer: 149.9
Knee Angle too low => Client: 133.2 vs Trainer: 149.9
Knee Angle too low => Client: 142.0 vs Trainer: 149.9
Knee Angle too low => Client: 148.6 vs Trainer: 149.9
Knee Angle too low => Client: 145.6 vs Trainer: 150.5
Knee Angle too low => Client: 145.8 vs Trainer: 150.5
Knee Angle too low => Client: 148.0 vs Trainer: 150.5
Knee Angle too low => Client: 147.8 vs Trainer: 152.9
Knee Angle too low => Client: 147.6 vs Trainer: 152.9
Knee Angle too low => Client: 149.3 vs Trainer: 152.9
Knee Angle too low => Client: 1